<a href="https://colab.research.google.com/github/MichalMichniak/Sieci-Neuronowe/blob/main/Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision
import torch
import torch.optim as optim
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)



100%|██████████| 26421880/26421880 [00:02<00:00, 12370475.21it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 212071.95it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:06<00:00, 696995.56it/s] 


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 24125449.15it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


In [4]:
print(train_dataloader.dataset.data.shape)


torch.Size([60000, 28, 28])


### Simple ANN

In [ ]:
class NN:
  def __init__(self):
    self.model = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Flatten(),
        nn.Linear(12544, 512),
        nn.ReLU(),
        nn.Linear(512, 64)
    ).to(device)

  def forward(self, x):
    return self.model(x)

  def validate_model(self, test_dataloader, device):
    self.model.eval()  # Ustaw model w tryb ewaluacji
    total = 0
    correct = 0

    with torch.no_grad():  # Wyłączenie obliczania gradientów
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = self.model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # Wybór klasy z najwyższym prawdopodobieństwem

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')


  def train_model(self, train_dataloader, test_dataloader, num_epochs=10, learning_rate=0.001):
    # Użycie GPU jeśli dostępne
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Zdefiniowanie optymalizatora i funkcji straty
    optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()  # Użycie CrossEntropyLoss dla klasyfikacji

    for epoch in range(num_epochs):
        self.model.train()  # Ustaw model w tryb treningowy
        running_loss = 0.0

        for inputs, labels in tqdm(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zerowanie gradientów
            optimizer.zero_grad()

            # Forward pass
            outputs = self.model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Walidacja na końcu każdej epoki
        self.validate_model( test_dataloader, device)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_dataloader):.4f}")




In [ ]:
print(NN().model)

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Flatten(start_dim=1, end_dim=-1)
  (6): Linear(in_features=12544, out_features=512, bias=True)
  (7): ReLU()
  (8): Linear(in_features=512, out_features=64, bias=True)
)


In [ ]:

model = NN()
model.train_model(train_dataloader, test_dataloader, num_epochs=10, learning_rate=0.001)

100%|██████████| 938/938 [00:12<00:00, 76.70it/s]


Validation Accuracy: 88.64%
Epoch [1/10], Loss: 0.3859


100%|██████████| 938/938 [00:12<00:00, 73.71it/s]


Validation Accuracy: 90.38%
Epoch [2/10], Loss: 0.2253


100%|██████████| 938/938 [00:12<00:00, 77.47it/s]


...

Validation Accuracy: 91.81%
Epoch [8/10], Loss: 0.0324


100%|██████████| 938/938 [00:12<00:00, 77.17it/s]


Validation Accuracy: 92.00%
Epoch [9/10], Loss: 0.0268


100%|██████████| 938/938 [00:12<00:00, 77.31it/s]


Validation Accuracy: 92.43%
Epoch [10/10], Loss: 0.0217


### Autoencoder

In [5]:
class Autoenc:
  def __init__(self):
    self.encoder = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Flatten(),
        nn.Linear(12544, 512),
        nn.ReLU(),
        nn.Linear(512, 12)
    ).to(device)

    self.decoder = nn.Sequential(
        nn.Linear(12, 512),
        nn.ReLU(),
        nn.Linear(512, 12544),
        nn.ReLU(),
        nn.Unflatten(1, (64, 14, 14)),
        nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
        nn.Sigmoid()

    ).to(device)

  def forward(self, x):
    return self.decoder(self.encoder(x))

  def validate_model(self, test_dataloader, device):
    self.encoder.eval()  # Ustaw model w tryb ewaluacji
    self.decoder.eval()

    running_loss = 0

    with torch.no_grad():  # Wyłączenie obliczania gradientów
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = self.forward(inputs)
            running_loss += nn.MSELoss()(outputs, inputs).item()


    meanloss = running_loss / len(test_dataloader)
    print(f'Validation Loss: {meanloss:.2f}')


  def train_model(self, train_dataloader, test_dataloader, num_epochs=10, learning_rate=0.001):
    # Użycie GPU jeśli dostępne
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Zdefiniowanie optymalizatora i funkcji straty
    optimizer = optim.Adam(list(self.encoder.parameters()) + list(self.decoder.parameters()), lr=learning_rate)
    criterion = nn.MSELoss()  # Użycie CrossEntropyLoss dla klasyfikacji

    for epoch in range(num_epochs):
        self.encoder.train()  # Ustaw model w tryb treningowy
        self.decoder.train()
        running_loss = 0.0

        for inputs, labels in tqdm(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zerowanie gradientów
            optimizer.zero_grad()

            # Forward pass
            outputs = self.forward(inputs)
            loss = nn.MSELoss()(outputs, inputs)

            # Backward pass
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Walidacja na końcu każdej epoki
        self.validate_model( test_dataloader, device)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_dataloader):.4f}")

In [6]:
model = Autoenc()
model.train_model(train_dataloader, test_dataloader, num_epochs=10, learning_rate=0.001)

100%|██████████| 938/938 [00:17<00:00, 54.11it/s]


Validation Loss: 0.01
Epoch [1/10], Loss: 0.0187


100%|██████████| 938/938 [00:15<00:00, 59.13it/s]


Validation Loss: 0.01
Epoch [2/10], Loss: 0.0119


100%|██████████| 938/938 [00:15<00:00, 60.66it/s]


...

Validation Loss: 0.01
Epoch [8/10], Loss: 0.0094


100%|██████████| 938/938 [00:15<00:00, 60.75it/s]


Validation Loss: 0.01
Epoch [9/10], Loss: 0.0093


100%|██████████| 938/938 [00:16<00:00, 55.64it/s]


Validation Loss: 0.01
Epoch [10/10], Loss: 0.0091


In [ ]:
import matplotlib.pyplot as plt

random_vector = torch.randn(1, 12).to(device)

decoded_image = model.decoder(random_vector)

decoded_image = decoded_image.reshape(28, 28).detach().cpu().numpy()

plt.imshow(decoded_image, cmap='gray')
plt.show()